In [1]:
import string
import requests
import secrets
import hashlib
import base64
import urllib.parse
import webbrowser
import pprint
import pandas as pd
import json
from IPython.display import Image

In [2]:
# APIアクセスに必要なキー情報
client_id = "8ba3fe1113b04b63b5e2d06d19b3e628"
redirect_uri = "http://localhost/callback/"

## 認証処理

In [3]:
# コードベリファイアの生成
accept_char = string.ascii_letters + string.digits + "_.-~"  # 許可される文字列
code_verifier = "".join(secrets.choice(accept_char) for i in range(128))
print(f"code verifier: {code_verifier}")

# コードチャレンジの生成
m = hashlib.sha256()
m.update(code_verifier.encode())
hashed_code_verifier = m.digest()
code_challenge = base64.urlsafe_b64encode(hashed_code_verifier).decode().replace('=', '')
print(f"code challenge: {code_challenge}")

code verifier: 8HA~wbHu.Elc-pEP5MECnpN4SxAGvZxbvQkoKyIwmQxUc30cSp4Fag_oXyoHvkQjCMSrm3orIfJTV~55LoGBB.1c3IKSgUfdTWajmUd0Qc6CAAUISHLM3Jq7dYqL04go
code challenge: P2lYDcfy2LQEBOQLU4SEaRQzMqlFR_lLF1g_EYXNQuA


In [4]:
# 認可のエンドポイント
authorization_url = "https://accounts.spotify.com/authorize"

# ステートパラメータの生成
state = "".join(secrets.choice(string.ascii_letters + string.digits) for ii in range(14))
print(f"state: {state}")

# 認可処理のパラメータ生成
authorization_params = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": redirect_uri,
    "code_challenge_method": "S256",
    "code_challenge": code_challenge,
    "state": state
}

state: ZzyxkGf9kkscEa


### 認可手続き

ブラウザで認可URLを開き、Spotify 認証後に API の認可をする

In [5]:
url = authorization_url + "?" + urllib.parse.urlencode(authorization_params)
print(url)
webbrowser.open(url)

https://accounts.spotify.com/authorize?client_id=8ba3fe1113b04b63b5e2d06d19b3e628&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%2Fcallback%2F&code_challenge_method=S256&code_challenge=P2lYDcfy2LQEBOQLU4SEaRQzMqlFR_lLF1g_EYXNQuA&state=ZzyxkGf9kkscEa


True

認可後にlocalhostへリダイレクトするので、そのURLを入力

In [6]:
redirect_url = input()
print(redirect_url)

http://localhost/callback/?code=AQBwRimBJfHgXMZ4ZbFEFsk9QDxhijuBXVYXJPL02P0CFoO0rJYgUhr8LY759Iy6YwBMT-1cMJLkX6J82xcRm-EAx0oCY3KkhMk6I__ny9MoPh1RHJ4ZOWEpnSZ_QFFOcIPOeNJibZMUKP1x-Ra-BYoMcbdMf9nwlBOGAb_0LT1txp3DBIMjxcBCrlOkfsukDmKuAw1n3VpOb7iRChJJzlt_hxRAuMgVWzsg31DVqgGB&state=ZzyxkGf9kkscEa
http://localhost/callback/?code=AQBwRimBJfHgXMZ4ZbFEFsk9QDxhijuBXVYXJPL02P0CFoO0rJYgUhr8LY759Iy6YwBMT-1cMJLkX6J82xcRm-EAx0oCY3KkhMk6I__ny9MoPh1RHJ4ZOWEpnSZ_QFFOcIPOeNJibZMUKP1x-Ra-BYoMcbdMf9nwlBOGAb_0LT1txp3DBIMjxcBCrlOkfsukDmKuAw1n3VpOb7iRChJJzlt_hxRAuMgVWzsg31DVqgGB&state=ZzyxkGf9kkscEa


In [7]:
# 認可処理の検証
redirect_url_parse_result = urllib.parse.urlparse(redirect_url)
redirect_url_param_dict = urllib.parse.parse_qs(redirect_url_parse_result.query)
pprint.pprint(redirect_url_param_dict)

# CSRFのために付与した state の検証
print(f"stateの一致: {state == redirect_url_param_dict['state'][0]}")

# 認可コード
authorization_code = redirect_url_param_dict['code'][0]
print(f"authorization code: {authorization_code}")

{'code': ['AQBwRimBJfHgXMZ4ZbFEFsk9QDxhijuBXVYXJPL02P0CFoO0rJYgUhr8LY759Iy6YwBMT-1cMJLkX6J82xcRm-EAx0oCY3KkhMk6I__ny9MoPh1RHJ4ZOWEpnSZ_QFFOcIPOeNJibZMUKP1x-Ra-BYoMcbdMf9nwlBOGAb_0LT1txp3DBIMjxcBCrlOkfsukDmKuAw1n3VpOb7iRChJJzlt_hxRAuMgVWzsg31DVqgGB'],
 'state': ['ZzyxkGf9kkscEa']}
stateの一致: True
authorization code: AQBwRimBJfHgXMZ4ZbFEFsk9QDxhijuBXVYXJPL02P0CFoO0rJYgUhr8LY759Iy6YwBMT-1cMJLkX6J82xcRm-EAx0oCY3KkhMk6I__ny9MoPh1RHJ4ZOWEpnSZ_QFFOcIPOeNJibZMUKP1x-Ra-BYoMcbdMf9nwlBOGAb_0LT1txp3DBIMjxcBCrlOkfsukDmKuAw1n3VpOb7iRChJJzlt_hxRAuMgVWzsg31DVqgGB


## アクセストークンの取得処理

In [8]:
# アクセストークン取得リクエストの生成
request_access_token_url = "https://accounts.spotify.com/api/token"
request_access_token_headers = {'Content-Type': 'application/x-www-form-urlencoded'}
request_access_token_params_dict = {
    "client_id": client_id,
    "grant_type": "authorization_code",
    "code": authorization_code,
    "redirect_uri": redirect_uri,
    "code_verifier": code_verifier
}
pprint.pprint(request_access_token_params_dict)

{'client_id': '8ba3fe1113b04b63b5e2d06d19b3e628',
 'code': 'AQBwRimBJfHgXMZ4ZbFEFsk9QDxhijuBXVYXJPL02P0CFoO0rJYgUhr8LY759Iy6YwBMT-1cMJLkX6J82xcRm-EAx0oCY3KkhMk6I__ny9MoPh1RHJ4ZOWEpnSZ_QFFOcIPOeNJibZMUKP1x-Ra-BYoMcbdMf9nwlBOGAb_0LT1txp3DBIMjxcBCrlOkfsukDmKuAw1n3VpOb7iRChJJzlt_hxRAuMgVWzsg31DVqgGB',
 'code_verifier': '8HA~wbHu.Elc-pEP5MECnpN4SxAGvZxbvQkoKyIwmQxUc30cSp4Fag_oXyoHvkQjCMSrm3orIfJTV~55LoGBB.1c3IKSgUfdTWajmUd0Qc6CAAUISHLM3Jq7dYqL04go',
 'grant_type': 'authorization_code',
 'redirect_uri': 'http://localhost/callback/'}


In [9]:
# アクセストークンの取得
request_access_token_response = requests.post(request_access_token_url,
                                              data=request_access_token_params_dict,
                                              headers=request_access_token_headers)

pprint.pprint(vars(request_access_token_response))

{'_content': b'{"access_token":"BQAwb3YC9lox4U3RYtqw7V8U6QMAm3wrx_f77EpTT9kg8qH'
             b'0jldiKX2T0XiMNaKdz9iwS1E6rD9QM6ebhZiDXqsMH51A41Noa2G5i-od5HwcAVH'
             b'2OaTFx2zTeFwrhKoYtFQ5cZuVfZAqb2bTa5p5DqLq0rIC44rgMTwpQQ","token_'
             b'type":"Bearer","expires_in":3600,"refresh_token":"AQAd00GWuds58B'
             b'TZMy4F0gmWMsP2r91AD8-ToQWGUvDskDbXb1y39wSbc41N_pdbn4L63I2WeiFGhl'
             b'zhP04M38MrQ2XJ8coQC3Fk7lgUTaDaVc5DMC2x1jFhpmUaXLdOJSM"}',
 '_content_consumed': True,
 '_next': None,
 'connection': <requests.adapters.HTTPAdapter object at 0x000002A2F532D5E0>,
 'cookies': <RequestsCookieJar[Cookie(version=0, name='sp_aor', value='Angular', port=None, port_specified=False, domain='.accounts.spotify.com', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1644105143, discard=False, comment=None, comment_url=None, rest={'SameSite': 'None'}, rfc2109=True), Cookie(version=0, name='sp_tr', value='false', port=N

In [10]:
# 取得したトークン情報を変数に格納
response_json = request_access_token_response.json()
access_token = response_json['access_token']
token_type = response_json['token_type']
# scope = response_json['scope']
expires_in = response_json['expires_in']
refresh_token = response_json['refresh_token']

In [11]:
# リフレッシュトークンからトークンの取得
refreshed_access_token_url = "https://accounts.spotify.com/api/token"
refreshed_access_token_headers = {'Content-Type': 'application/x-www-form-urlencoded'}
refreshed_access_token_data_dict = {
    "grant_type": "refresh_token",
    "refresh_token": refresh_token,
    "client_id": client_id
}
refreshed_access_token_response = requests.post(refreshed_access_token_url,
                                                data=refreshed_access_token_data_dict,
                                                headers=refreshed_access_token_headers)
# pprint.pprint(vars(request_access_token_response.request))
pprint.pprint(vars(request_access_token_response))

{'_content': b'{"access_token":"BQAwb3YC9lox4U3RYtqw7V8U6QMAm3wrx_f77EpTT9kg8qH'
             b'0jldiKX2T0XiMNaKdz9iwS1E6rD9QM6ebhZiDXqsMH51A41Noa2G5i-od5HwcAVH'
             b'2OaTFx2zTeFwrhKoYtFQ5cZuVfZAqb2bTa5p5DqLq0rIC44rgMTwpQQ","token_'
             b'type":"Bearer","expires_in":3600,"refresh_token":"AQAd00GWuds58B'
             b'TZMy4F0gmWMsP2r91AD8-ToQWGUvDskDbXb1y39wSbc41N_pdbn4L63I2WeiFGhl'
             b'zhP04M38MrQ2XJ8coQC3Fk7lgUTaDaVc5DMC2x1jFhpmUaXLdOJSM"}',
 '_content_consumed': True,
 '_next': None,
 'connection': <requests.adapters.HTTPAdapter object at 0x000002A2F532D5E0>,
 'cookies': <RequestsCookieJar[Cookie(version=0, name='sp_aor', value='Angular', port=None, port_specified=False, domain='.accounts.spotify.com', domain_specified=True, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1644105143, discard=False, comment=None, comment_url=None, rest={'SameSite': 'None'}, rfc2109=True), Cookie(version=0, name='sp_tr', value='false', port=N

In [12]:
# 取得したトークン情報を変数に格納
response_json = refreshed_access_token_response.json()
access_token = response_json['access_token']
token_type = response_json['token_type']
# scope = response_json['scope']
expires_in = response_json['expires_in']
refresh_token = response_json['refresh_token']

print(f"access token: {access_token}")

access token: BQBYzEVXrJwR6mxMewukz4QQdorstJW2oCojN0urqZFg3A7R69CBBhLy172iKHBAgvXUy73yly0ld9R5VDkgPOlyaIFyXmLvdStckmBjE7hLFLQ4Hdz7uih2jsEMOMceVmkM9xpWYjP0T2Y-tQztfFqCrXlu1XVb45WELQ


In [13]:
# API に必要な HTTP ヘッダー情報を生成
api_header_dict = {
    "Authorization": f"Bearer {access_token}"
}
pprint.pprint(api_header_dict)

{'Authorization': 'Bearer '
                  'BQBYzEVXrJwR6mxMewukz4QQdorstJW2oCojN0urqZFg3A7R69CBBhLy172iKHBAgvXUy73yly0ld9R5VDkgPOlyaIFyXmLvdStckmBjE7hLFLQ4Hdz7uih2jsEMOMceVmkM9xpWYjP0T2Y-tQztfFqCrXlu1XVb45WELQ'}


## アーティスト情報の取得

In [14]:
search_api_url = "https://api.spotify.com/v1/search"
query = input()
item_type = "artist"  # album , artist, playlist, track, show and episode
market = "JP"

response = requests.get(search_api_url,
                        headers=api_header_dict,
                        params={
                            "q": query,
                            "type": item_type,
                            "market": market
                        })
response_json = response.json()
pprint.pprint(response_json)


緑黄色社会
{'artists': {'href': 'https://api.spotify.com/v1/search?query=%E7%B7%91%E9%BB%84%E8%89%B2%E7%A4%BE%E4%BC%9A&type=artist&market=JP&offset=0&limit=20',
             'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4SJ7qRgJYNXB9Yttzs4aSa'},
                        'followers': {'href': None, 'total': 401476},
                        'genres': ['j-pop'],
                        'href': 'https://api.spotify.com/v1/artists/4SJ7qRgJYNXB9Yttzs4aSa',
                        'id': '4SJ7qRgJYNXB9Yttzs4aSa',
                        'images': [{'height': 640,
                                    'url': 'https://i.scdn.co/image/ab6761610000e5ebf4abdd5f688bb55f051781bc',
                                    'width': 640},
                                   {'height': 320,
                                    'url': 'https://i.scdn.co/image/ab67616100005174f4abdd5f688bb55f051781bc',
                                    'width': 320},
                                   {'heigh

In [15]:
# 検索したアーティストのIDを取得
artist_spotify_id = response_json['artists']['items'][0]['id']
print(artist_spotify_id)

4SJ7qRgJYNXB9Yttzs4aSa


## アーティストのトップ曲を取得

In [16]:
top_track_url = f"https://api.spotify.com/v1/artists/{artist_spotify_id}/top-tracks"
print(top_track_url)
response = requests.get(top_track_url,
                        headers=api_header_dict,
                        params={
                            "market": market
                        })
top_tracks_response =response.json()
pprint.pprint(top_tracks_response)
tracks = {track['id'] : track for track in top_tracks_response['tracks']}

with open("./top_tracks.json", "w") as f:
    json.dump(tracks, f)

https://api.spotify.com/v1/artists/4SJ7qRgJYNXB9Yttzs4aSa/top-tracks
{'tracks': [{'album': {'album_type': 'single',
                       'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4SJ7qRgJYNXB9Yttzs4aSa'},
                                    'href': 'https://api.spotify.com/v1/artists/4SJ7qRgJYNXB9Yttzs4aSa',
                                    'id': '4SJ7qRgJYNXB9Yttzs4aSa',
                                    'name': 'Ryokuoushoku Shakai',
                                    'type': 'artist',
                                    'uri': 'spotify:artist:4SJ7qRgJYNXB9Yttzs4aSa'}],
                       'external_urls': {'spotify': 'https://open.spotify.com/album/1a6MAP1V8kZfvmLFzhn0QG'},
                       'href': 'https://api.spotify.com/v1/albums/1a6MAP1V8kZfvmLFzhn0QG',
                       'id': '1a6MAP1V8kZfvmLFzhn0QG',
                       'images': [{'height': 640,
                                   'url': 'https://i.scdn.co/image/ab67616

In [17]:
# トップトラックの Spotify ID の取得
top_track_ids = [track['id'] for track in top_tracks_response['tracks']]
print(top_track_ids)

['6IO5nn84TKArsi3cjpIqaD', '5K1m4aaPCxwnm9SKlWW1vh', '2yOUBePdX7afs41FAy1XD7', '3oy9Y621WDsAXSAwJKSSZV', '0O4YelF6Zneq7sk1Ye73N2', '1oE30MDSjSxTYTOOkll9Kt', '7HPzdAV66KujBIhgs6eLCM', '64ztvh9KDjnGR07r1bUTnz', '6hVRV1wXUdJ86FywMBBeIa', '3Nu7UbcaJuaZLBKN3yGFjf']


## トラックのオーディオ分析を取得する

> Get a low-level audio analysis for a track in the Spotify catalog. The audio analysis describes the track’s structure and musical content, including rhythm, pitch, and timbre.
> Spotifyカタログのトラックの低レベルのオーディオ分析を取得します。オーディオ分析では、トラックの構造と、リズム、ピッチ、音色などの音楽コンテンツについて説明します。

In [18]:
audio_analysis_base_url = "https://api.spotify.com/v1/audio-analysis"
audio_analysis = {}
for id in top_track_ids:
    url = f"{audio_analysis_base_url}/{id}"
    response = requests.get(url,
                        headers=api_header_dict)
    audio_analysis[id] = response.json()
first_key = next(iter(audio_analysis))

pprint.pprint(audio_analysis[first_key])
with open("./audio_analysis.json", "w") as f:
    json.dump(audio_analysis, f)

{'bars': [{'confidence': 0.574, 'duration': 1.74302, 'start': 2.32838},
          {'confidence': 0.221, 'duration': 1.74078, 'start': 4.0714},
          {'confidence': 0.239, 'duration': 1.7385, 'start': 5.81219},
          {'confidence': 0.285, 'duration': 1.74472, 'start': 7.55069},
          {'confidence': 0.518, 'duration': 1.74001, 'start': 9.29541},
          {'confidence': 0.607, 'duration': 1.73058, 'start': 11.03542},
          {'confidence': 0.82, 'duration': 1.72223, 'start': 12.76599},
          {'confidence': 0.539, 'duration': 1.74035, 'start': 14.48822},
          {'confidence': 0.328, 'duration': 1.76044, 'start': 16.22857},
          {'confidence': 0.309, 'duration': 1.73114, 'start': 17.98901},
          {'confidence': 0.26, 'duration': 1.74182, 'start': 19.72015},
          {'confidence': 0.162, 'duration': 1.74244, 'start': 21.46197},
          {'confidence': 0.161, 'duration': 1.73997, 'start': 23.20441},
          {'confidence': 0.401, 'duration': 1.73756, 'start'

                          77.573,
                          68.461,
                          40.795,
                          31.606,
                          26.081,
                          -17.795,
                          7.231,
                          17.16,
                          -11.155,
                          -10.253,
                          -3.675]},
              {'confidence': 0.703,
               'duration': 0.19723,
               'loudness_end': 0.0,
               'loudness_max': -7.791,
               'loudness_max_time': 0.08278,
               'loudness_start': -16.597,
               'pitches': [0.425,
                           0.115,
                           0.134,
                           0.386,
                           0.469,
                           0.766,
                           0.252,
                           1.0,
                           0.936,
                           0.226,
                           0.056,
                 

               'pitches': [0.207,
                           0.271,
                           0.178,
                           0.493,
                           0.934,
                           1.0,
                           0.305,
                           0.082,
                           0.105,
                           0.083,
                           0.142,
                           0.253],
               'start': 24.67719,
               'timbre': [56.02,
                          111.707,
                          26.026,
                          -0.629,
                          -12.074,
                          -9.452,
                          -6.568,
                          4.087,
                          -9.696,
                          6.009,
                          -8.555,
                          -0.853]},
              {'confidence': 0.642,
               'duration': 0.21986,
               'loudness_end': 0.0,
               'loudness_max': -0.8,
     

               'pitches': [0.444,
                           0.308,
                           0.203,
                           0.291,
                           0.728,
                           0.236,
                           0.283,
                           0.296,
                           0.764,
                           0.974,
                           1.0,
                           0.71],
               'start': 42.55678,
               'timbre': [56.391,
                          51.714,
                          -1.505,
                          55.474,
                          -27.338,
                          -15.745,
                          0.923,
                          -15.395,
                          -11.144,
                          -5.658,
                          -39.693,
                          -3.23]},
              {'confidence': 0.732,
               'duration': 0.16068,
               'loudness_end': 0.0,
               'loudness_max': -3.373,


               'loudness_max_time': 0.03149,
               'loudness_start': -7.821,
               'pitches': [0.212,
                           0.347,
                           0.737,
                           1.0,
                           0.568,
                           0.389,
                           0.178,
                           0.079,
                           0.143,
                           0.14,
                           0.184,
                           0.139],
               'start': 59.24499,
               'timbre': [57.341,
                          111.552,
                          -10.063,
                          2.972,
                          -2.334,
                          -27.598,
                          -22.318,
                          -9.875,
                          -7.888,
                          -10.221,
                          3.937,
                          14.352]},
              {'confidence': 0.34,
               'duration':

...

              {'confidence': 0.768,
               'duration': 0.1381,
               'loudness_end': 0.0,
               'loudness_max': -1.457,
               'loudness_max_time': 0.0704,
               'loudness_start': -10.744,
               'pitches': [0.462,
                           0.508,
                           0.224,
                           0.153,
                           0.161,
                           0.269,
                           0.189,
                           0.318,
                           1.0,
                           0.35,
                           0.24,
                           0.235],
               'start': 174.4239,
               'timbre': [54.323,
                          -46.567,
                          75.52,
                          -69.897,
                          10.002,
                          -13.404,
                          -5.272,
                          18.967,
                          0.744,
                      

                           0.318],
               'start': 191.4195,
               'timbre': [58.607,
                          83.047,
                          29.128,
                          47.192,
                          -29.808,
                          -17.221,
                          -14.692,
                          -0.591,
                          -11.22,
                          -2.174,
                          -17.645,
                          -5.083]},
              {'confidence': 0.705,
               'duration': 0.20245,
               'loudness_end': 0.0,
               'loudness_max': 0.634,
               'loudness_max_time': 0.02753,
               'loudness_start': -6.526,
               'pitches': [0.047,
                           0.068,
                           0.072,
                           0.037,
                           0.043,
                           0.124,
                           0.285,
                           1.0,
               

                           0.069,
                           0.41,
                           0.054,
                           0.028,
                           0.06,
                           0.167,
                           1.0,
                           0.544],
               'start': 210.3267,
               'timbre': [55.402,
                          92.889,
                          8.947,
                          -8.076,
                          -3.168,
                          -3.505,
                          13.315,
                          14.226,
                          -8.877,
                          12.129,
                          -3.605,
                          -6.547]},
              {'confidence': 0.44,
               'duration': 0.22063,
               'loudness_end': 0.0,
               'loudness_max': -2.014,
               'loudness_max_time': 0.01805,
               'loudness_start': -8.634,
               'pitches': [0.163,
                      

                          17.287,
                          2.169,
                          4.678,
                          -17.738,
                          12.765,
                          18.221,
                          -14.855,
                          -11.12,
                          -5.769,
                          -2.378]},
              {'confidence': 0.758,
               'duration': 0.66181,
               'loudness_end': 0.0,
               'loudness_max': 0.633,
               'loudness_max_time': 0.05113,
               'loudness_start': -9.474,
               'pitches': [0.25,
                           0.106,
                           0.107,
                           0.132,
                           0.144,
                           1.0,
                           0.289,
                           0.182,
                           0.187,
                           0.242,
                           0.232,
                           0.193],
               'star

            {'confidence': 0.47, 'duration': 0.21637, 'start': 217.76819},
            {'confidence': 0.45, 'duration': 0.21656, 'start': 217.98456},
            {'confidence': 0.45, 'duration': 0.21656, 'start': 218.20113},
            {'confidence': 0.335, 'duration': 0.21695, 'start': 218.41768},
            {'confidence': 0.335, 'duration': 0.21695, 'start': 218.63461},
            {'confidence': 0.282, 'duration': 0.21617, 'start': 218.85156},
            {'confidence': 0.282, 'duration': 0.21617, 'start': 219.06773},
            {'confidence': 0.277, 'duration': 0.21636, 'start': 219.2839},
            {'confidence': 0.277, 'duration': 0.21636, 'start': 219.50026},
            {'confidence': 0.332, 'duration': 0.21675, 'start': 219.71661},
            {'confidence': 0.332, 'duration': 0.21675, 'start': 219.93336},
            {'confidence': 0.395, 'duration': 0.21656, 'start': 220.15012},
            {'confidence': 0.395, 'duration': 0.21656, 'start': 220.36667},
            {'co

## 画面表示用のDataFrame作成

In [19]:
df = pd.DataFrame([[tracks[track]['name'], tracks[track]['artists'][0]['name'], tracks[track]['album']['name']] for track in tracks],
                  columns = ['曲名', "アーティスト", "アルバム"],
                  index= [track for track in tracks])
df

曲名               アーティスト        アルバム
6IO5nn84TKArsi3cjpIqaD       Mela!  Ryokuoushoku Shakai       Mela!
5K1m4aaPCxwnm9SKlWW1vh  Shout Baby  Ryokuoushoku Shakai  Shout Baby
2yOUBePdX7afs41FAy1XD7   Landscape  Ryokuoushoku Shakai       Actor
3oy9Y621WDsAXSAwJKSSZV   ずっとずっとずっと  Ryokuoushoku Shakai       Actor
0O4YelF6Zneq7sk1Ye73N2      キャラクター  Ryokuoushoku Shakai       Actor
1oE30MDSjSxTYTOOkll9Kt      LITMUS  Ryokuoushoku Shakai       Actor
7HPzdAV66KujBIhgs6eLCM      たとえたとえ  Ryokuoushoku Shakai       Actor
64ztvh9KDjnGR07r1bUTnz    sabotage  Ryokuoushoku Shakai    sabotage
6hVRV1wXUdJ86FywMBBeIa     LADYBUG  Ryokuoushoku Shakai          結証
3Nu7UbcaJuaZLBKN3yGFjf          結証  Ryokuoushoku Shakai       Actor

In [20]:
# 全体テンポの追加
df['全体的なテンポ'] = [audio_analysis[k]['track']['tempo'] for k in df.index.values.tolist()]
df

曲名               アーティスト        アルバム  全体的なテンポ
6IO5nn84TKArsi3cjpIqaD       Mela!  Ryokuoushoku Shakai       Mela!  137.967
5K1m4aaPCxwnm9SKlWW1vh  Shout Baby  Ryokuoushoku Shakai  Shout Baby  180.000
2yOUBePdX7afs41FAy1XD7   Landscape  Ryokuoushoku Shakai       Actor  130.011
3oy9Y621WDsAXSAwJKSSZV   ずっとずっとずっと  Ryokuoushoku Shakai       Actor  127.976
0O4YelF6Zneq7sk1Ye73N2      キャラクター  Ryokuoushoku Shakai       Actor  124.025
1oE30MDSjSxTYTOOkll9Kt      LITMUS  Ryokuoushoku Shakai       Actor   95.981
7HPzdAV66KujBIhgs6eLCM      たとえたとえ  Ryokuoushoku Shakai       Actor  128.076
64ztvh9KDjnGR07r1bUTnz    sabotage  Ryokuoushoku Shakai    sabotage  161.978
6hVRV1wXUdJ86FywMBBeIa     LADYBUG  Ryokuoushoku Shakai          結証  123.985
3Nu7UbcaJuaZLBKN3yGFjf          結証  Ryokuoushoku Shakai       Actor   99.970

In [21]:
pitch_class = {
    0: 'C (also B♯, D♭♭) ',
    1: 'C♯, D♭ (also B♯♯) ',
    2: 'D (also C♯♯ sharp, E♭♭) ',
    3: 'D♯, E♭ (also F♭♭) ',
    4: 'E (also D♯♯, F♭) ',
    5: 'F (also E♯, G♭♭) ',
    6: 'F♯, G♭ (also E♯♯) ',
    7: 'G (also F♯♯, A♭♭) ',
    8: 'G♯, A♭',
    9: 'A (also G♯♯, B♭♭) ',
    10: 'A♯, B♭ (also C♭♭) ',
    11: 'B (also A♯♯, C♭) ',
}

In [22]:
# 全体的なキーの追加
df['全体的なキー'] = [pitch_class[audio_analysis[k]['track']['key']] for k in df.index.values.tolist()]
df

曲名               アーティスト        アルバム  全体的なテンポ  \
6IO5nn84TKArsi3cjpIqaD       Mela!  Ryokuoushoku Shakai       Mela!  137.967   
5K1m4aaPCxwnm9SKlWW1vh  Shout Baby  Ryokuoushoku Shakai  Shout Baby  180.000   
2yOUBePdX7afs41FAy1XD7   Landscape  Ryokuoushoku Shakai       Actor  130.011   
3oy9Y621WDsAXSAwJKSSZV   ずっとずっとずっと  Ryokuoushoku Shakai       Actor  127.976   
0O4YelF6Zneq7sk1Ye73N2      キャラクター  Ryokuoushoku Shakai       Actor  124.025   
1oE30MDSjSxTYTOOkll9Kt      LITMUS  Ryokuoushoku Shakai       Actor   95.981   
7HPzdAV66KujBIhgs6eLCM      たとえたとえ  Ryokuoushoku Shakai       Actor  128.076   
64ztvh9KDjnGR07r1bUTnz    sabotage  Ryokuoushoku Shakai    sabotage  161.978   
6hVRV1wXUdJ86FywMBBeIa     LADYBUG  Ryokuoushoku Shakai          結証  123.985   
3Nu7UbcaJuaZLBKN3yGFjf          結証  Ryokuoushoku Shakai       Actor   99.970   

                                    全体的なキー  
6IO5nn84TKArsi3cjpIqaD              G♯, A♭  
5K1m4aaPCxwnm9SKlWW1vh  G (also F♯♯, A♭♭)   
2yOUBePdX7afs41FAy1XD7  C♯, D♭ (also B♯♯)   
3oy9Y621WDsAXSAwJKSSZV  A♯, B♭ (also C♭♭)   
0O4YelF6Zneq7sk1Ye73N2   F (also E♯, G♭♭)   
1oE30MDSjSxTYTOOkll9Kt  G (also F♯♯, A♭♭)   
7HPzdAV66KujBIhgs6eLCM  C♯, D♭ (also B♯♯)   
64ztvh9KDjnGR07r1bUTnz  D♯, E♭ (also F♭♭)   
6hVRV1wXUdJ86FywMBBeIa   E (also D♯♯, F♭)   
3Nu7UbcaJuaZLBKN3yGFjf  F♯, G♭ (also E♯♯)